In [2]:
import pandas as pd

In [ ]:
#Assumptions:
# 2 funds - adjust if more funds bought in the future
# monthly entries that contain fund name, date, units bought, units sold, and price per unit
# data covers the whole of the previous year, up until the point where the current_values_file gives data on the current number of units (which may be in the following year)


In [15]:
year_begin = '2021-01-01'
year_end = '2021-12-31'

In [41]:
transactions = pd.read_csv("data/legal_and_general_transactions.csv")
current_values_file = 'data/fund_units_legal_and_general_march2022.txt'

In [8]:
transactions.head()

,Date,Transaction description,Payments in,Payments out,Unit Investments,Fund name,Units bought,Units sold,Price per unit (p)
0,2022-03-02,Your Employer's regular contribution,731.53,0.00,731.53,L&G PMC 2045 - 2050 Target Date Fund 3,27.714,0.000,1319.78
1,2022-03-02,NaN,NaN,NaN,NaN,L&G PMC World (Ex-UK) Equity Index 3,59.452,0.000,615.23
2,2022-03-01,Management Charge,0.00,5.06,-5.06,L&G PMC 2045 - 2050 Target Date Fund 3,0.000,0.180,1321.17
3,2022-03-01,NaN,NaN,NaN,NaN,L&G PMC World (Ex-UK) Equity Index 3,0.000,0.432,620.29
4,2022-02-01,Your Employer's regular contribution,1585.00,0.00,1585.00,L&G PMC 2045 - 2050 Target Date Fund 3,58.578,0.000,1352.90


In [20]:
df = transactions[['Fund name', 'Units bought', 'Units sold', 'Price per unit (p)', 'Date']]

In [33]:
df = df.dropna(axis=0, how='any')

In [36]:
funds = df['Fund name'].unique()
print(funds)

['L&G PMC 2045 - 2050 Target Date Fund 3'
 'L&G PMC World (Ex-UK) Equity Index 3']


In [37]:
mapping_dict = {funds[i]:"fund_"+str(i) for i in range(len(funds))}

In [38]:
mapping_dict

{'L&G PMC 2045 - 2050 Target Date Fund 3': 'fund_0',
 'L&G PMC World (Ex-UK) Equity Index 3': 'fund_1'}

In [42]:
with open(current_values_file) as f:
    current_values = {str(k):float(v) for line in f for (k,v) in [line.strip().split(':')]}
print(current_values)

{'L&G PMC 2045 - 2050 Target Date Fund 3': 2164.626, 'L&G PMC World (Ex-UK) Equity Index 3': 5187.65}


In [43]:
#Now change fund names using the mapping_dict
df = df.replace(mapping_dict)

In [57]:
current_values = {mapping_dict[k]:v for (k,v) in current_values.items()}

In [ ]:
#Find number of units for each fund at the beginning of the year, and also at the end of the year. 
#Find increase or decrease in price for each fund, end of year compared with beginning of year. 

In [75]:
max_date = max(df[df.Date<=year_end]['Date'])
min_date = min(df[df.Date>=year_begin]['Date'])

In [62]:
fund0 = df[df['Fund name'] == 'fund_0'].reset_index(drop=True)
fund1 = df[df['Fund name'] == 'fund_1'].reset_index(drop=True)

In [76]:
fund0_future = fund0[fund0.Date>max_date]
fund0_units_eoy = current_values['fund_0']-sum(fund0_future['Units bought']) + sum(fund0_future['Units sold'])

fund1_future = fund1[fund1.Date>max_date]
fund1_units_eoy = current_values['fund_1']-sum(fund1_future['Units bought']) + sum(fund1_future['Units sold'])

In [77]:
fund1_units_eoy

5004.846

In [78]:
fund0_year = fund0[((fund0.Date>=min_date)&(fund0.Date<=max_date))]
fund1_year = fund1[((fund1.Date>=min_date)&(fund1.Date<=max_date))]

In [92]:
fund0_units_boy = fund0_units_eoy - (sum(fund0_year['Units bought']) - sum(fund0_year['Units sold']))
fund1_units_boy = fund1_units_eoy - (sum(fund1_year['Units bought']) - sum(fund1_year['Units sold']))

In [93]:
fund1_units_boy

3607.6919999999996

In [94]:
fund0_unit_increase = fund0_units_eoy - fund0_units_boy
fund1_unit_increase = fund1_units_eoy - fund1_units_boy

In [114]:
initial_price_fund0 = fund0_year[fund0_year.Date==min_date][:1]['Price per unit (p)'].values[0]
final_price_fund0 = fund0_year[fund0_year.Date==max_date][:1]['Price per unit (p)'].values[0]

initial_price_fund1 = fund1_year[fund1_year.Date==min_date][:1]['Price per unit (p)'].values[0]
final_price_fund1 = fund1_year[fund1_year.Date==max_date][:1]['Price per unit (p)'].values[0]

In [135]:
fund0_base_increase = fund0_units_boy*(final_price_fund0-initial_price_fund0)/100.0  #convert to pounds

In [136]:
fund1_base_increase = fund1_units_boy*(final_price_fund1-initial_price_fund1)/100.0  #convert to pounds

In [126]:
fund1_year['final_year_price'] = final_price_fund1/100
fund0_year['final_year_price'] = final_price_fund0/100
fund0_year['unit_price_pounds'] = fund0_year['Price per unit (p)']/100.0
fund1_year['unit_price_pounds'] = fund1_year['Price per unit (p)']/100.0

fund0_year['increase_eoy'] = fund0_year['Units bought']*(fund0_year['final_year_price']
                                                         -fund0_year['unit_price_pounds'])
fund1_year['increase_eoy'] = fund1_year['Units bought']*(fund1_year['final_year_price']
                                                         -fund1_year['unit_price_pounds'])

#Note: this is not including the sold values, so this is slightly incorrect. 

/tmp/ipykernel_68839/1551814744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund1_year['final_year_price'] = final_price_fund1/100
/tmp/ipykernel_68839/1551814744.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fund0_year['final_year_price'] = final_price_fund0/100
/tmp/ipykernel_68839/1551814744.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [127]:
fund0_bought_increase = sum(fund0_year['increase_eoy'])
fund1_bought_increase = sum(fund1_year['increase_eoy'])                    

In [138]:
total_fund0_increase = fund0_bought_increase+fund0_base_increase
total_fund1_increase = fund1_bought_increase+fund1_base_increase

In [141]:
total_fund0_increase+total_fund1_increase

6860.192233399999

In [150]:
sum(fund0_year['Units bought']*fund0_year['unit_price_pounds']) + sum(fund1_year['Units bought']*fund1_year['unit_price_pounds'])


17155.0137805

In [151]:
6860/0.727

9436.038514442916

In [152]:
17155/0.727

23596.973865199452

In [ ]:
# The above value matches the total sum of contributions for 2021